In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = "https://www.imdb.com/list/ls006444168/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Webscrape imdb movies from books list. 
movies_list = soup.find_all('div', class_='lister-list')

In [5]:
# Block no longer needed

# # Webscrape imdb movies from books list. 
# movies_list = soup.find_all('div', class_='lister-list')

# # Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

# title = movies_list[0].find('h3').find('a').text

# year = movies_list[0].find('span',class_='lister-item-year').text

# movie_length = movies_list[0].find('span',class_='runtime').text

# genre = movies_list[0].find('span',class_='genre').text

# # javascript for certificate: # <span class="certificate">R</span>
# certificate = movies_list[0].find('span',class_='certificate').text

# # javascript for rating: <span class="ipl-rating-star__rating">7.5</span>
# stars = movies_list[0].find('span',class_='ipl-rating-star__rating').text

# # javascript for movie gross:
# # <span class="text-muted">Gross:</span>
# # <span name="nv" data-value="18,335,230">$18.34M</span>

# # chatGPT   
# gross = movies_list[0].find('span', string='Gross:')

# if gross:
#     gross_amount = gross.find_next_sibling('span')['data-value']
# else:
#     gross_amount = 'N/A'
    
# gross_value = gross.find_next_sibling('span')['data-value']

# #********************Discard later?
# # # find the <p> tag with class 'text_muted text-small'
# ##p_tag = soup.find('p', class_='text-muted text-small')
# #p_tag = soup.find('span', class_='text-muted').text
# #p_tag = soup.find('span', name_='nv')


# # # find the <span> tag with name 'nv' inside the <p> tag
# #span_tag = p_tag.find('span', name='nv')

# # # extract the text content of the <span> tag.
# # text = span_tag.text.strip()

In [5]:
# Create empty lists to hold the scraped data
movies_data = []

def get_movietitle():
    while True:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        movies_list = soup.find_all('div', class_='lister-item-content')

        for movie in movies_list:
            movie_data = {}

            title = movie.find('h3').find('a').text
            
            movie_year = movie.find('span', class_='lister-item-year').text
            # Convert the year to a positive value
            movie_year = int(''.join(filter(str.isdigit, movie_year)))
            
            movie_len = movie.find('span', class_='runtime').text
            genre = movie.find('span', class_='genre').text.strip()

            try:
                certificate = movie.find('span', class_='certificate').text
            except AttributeError:
                certificate = 'N/A'

            stars = movie.find('span', class_='ipl-rating-star__rating').text

            gross = movie.find_all('span', string='Gross:')
            if gross:
                gross_value = gross[0].find_next_sibling('span')['data-value']
            else:
                gross_value = 'N/A'

            # Store the data in a dictionary
            movie_data['movie_title'] = title
            movie_data['year'] = movie_year
            movie_data['length'] = movie_len
            movie_data['movie_genre'] = genre
            movie_data['movie_rating'] = certificate
            movie_data['star_rating'] = stars
            movie_data['gross_amount'] = gross_value

            # Append the dictionary to the movies_data list
            movies_data.append(movie_data)

        # Check if there is a "Next" button
        next_button = browser.find_by_xpath('//a[@class="flat-button lister-page-next next-page"]')
        if next_button.is_empty():
            break
        else:
            next_button.first.click()
            time.sleep(2)  # Add a small delay before continuing to the next page

            # Wait for the page to refresh and find the "Next" button again
            browser.is_element_present_by_xpath('//a[@class="flat-button lister-page-next next-page"]', wait_time=10)

get_movietitle()

In [6]:
movies_data

[{'movie_title': '127 Hours',
  'year': 2010,
  'length': '94 min',
  'movie_genre': 'Biography, Drama',
  'movie_rating': 'R',
  'star_rating': '7.5',
  'gross_amount': '18,335,230'},
 {'movie_title': '100 Rifles',
  'year': 1969,
  'length': '110 min',
  'movie_genre': 'Adventure, Drama, War',
  'movie_rating': 'R',
  'star_rating': '6',
  'gross_amount': 'N/A'},
 {'movie_title': 'The Hunger Games',
  'year': 2012,
  'length': '142 min',
  'movie_genre': 'Action, Adventure, Sci-Fi',
  'movie_rating': 'PG-13',
  'star_rating': '7.2',
  'gross_amount': '408,010,692'},
 {'movie_title': 'The Hunger Games: Catching Fire',
  'year': 2013,
  'length': '146 min',
  'movie_genre': 'Action, Adventure, Sci-Fi',
  'movie_rating': 'PG-13',
  'star_rating': '7.5',
  'gross_amount': '424,668,047'},
 {'movie_title': 'A Beautiful Mind',
  'year': 2001,
  'length': '135 min',
  'movie_genre': 'Biography, Drama',
  'movie_rating': 'PG-13',
  'star_rating': '8.2',
  'gross_amount': '170,742,341'},
 {'mo

In [7]:
# No longer needed; 
# movies_fr_books_dict['movie_title'] = movie_title
# movies_fr_books_dict['year'] = year
# movies_fr_books_dict['movie_length'] = movie_length
# movies_fr_books_dict['movie_genre'] = movie_genre
# movies_fr_books_dict['movie_rating'] = movie_rating
# movies_fr_books_dict['star_rating'] = star_rating
# movies_fr_books_dict['gross_amount'] = gross_value

In [8]:
browser.quit()

In [9]:
import pandas as pd
moviesfrombooks_data = pd.DataFrame(movies_data)
moviesfrombooks_data.head(5)

,movie_title,year,length,movie_genre,movie_rating,star_rating,gross_amount
0,127 Hours,2010,94 min,"Biography, Drama",R,7.5,"18,335,230"
1,100 Rifles,1969,110 min,"Adventure, Drama, War",R,6,N/A
2,The Hunger Games,2012,142 min,"Action, Adventure, Sci-Fi",PG-13,7.2,"408,010,692"
3,The Hunger Games: Catching Fire,2013,146 min,"Action, Adventure, Sci-Fi",PG-13,7.5,"424,668,047"
4,A Beautiful Mind,2001,135 min,"Biography, Drama",PG-13,8.2,"170,742,341"


In [10]:
#Identify datatype.
moviesfrombooks_data.dtypes

movie_title     object
year             int64
length          object
movie_genre     object
movie_rating    object
star_rating     object
gross_amount    object
dtype: object

In [11]:
# Display random sample of 10 rows from dataframe, moviesfrombooks_data

moviesfrombooks_data.sample(n=10)

,movie_title,year,length,movie_genre,movie_rating,star_rating,gross_amount
18,The Notebook,2004,123 min,"Drama, Romance",PG-13,7.8,"81,001,787"
146,Percy Jackson & the Olympians: The Lightning T...,2010,118 min,"Adventure, Family, Fantasy",PG,5.8,"88,761,720"
84,Green Zone,2010,115 min,"Action, Drama, Thriller",R,6.8,"35,053,660"
14,Message in a Bottle,1999,126 min,"Drama, Romance",PG-13,6.2,"52,880,016"
69,Misery,1990,107 min,"Drama, Thriller",R,7.8,"61,276,872"
99,Revolutionary Road,2008,119 min,"Drama, Romance",R,7.3,"22,911,480"
122,The Girl with the Dragon Tattoo,2011,158 min,"Crime, Drama, Mystery",R,7.8,"102,515,793"
76,Ballet Shoes,2007,85 min,"Drama, Family, Musical",PG,6.6,N/A
11,About a Boy,2002,101 min,"Comedy, Drama, Romance",PG-13,7.1,"41,385,278"
56,Stardust,2007,127 min,"Adventure, Family, Fantasy",PG-13,7.6,"38,634,938"


In [12]:
# Block no longer needed
# moviesfrombooks_data.iloc[0]['genre'].split(', ')

In [15]:
# Block no longer needed
# extract digits only from `year` column
# moviesfrombooks_data['year'] = moviesfrombooks_data['year'].str.extract(r'\((\d+)\)')

In [18]:
# Block no longer needed
# extract digits only from `movie_length` column
# moviesfrombooks_data['length'] = moviesfrombooks_data['length'].str.extract(r'(\d+)\s*min')

In [19]:
# Block no longer needed
# split genre text string into a list of genres
# moviesfrombooks_data['genre'] = moviesfrombooks_data['genre'].str.split(', ')

In [20]:
# # Block no longer needed
# moviesfrombooks_data = pd.get_dummies(moviesfrombooks_data, columns = ['rating'])

In [21]:
# Block no longer needed not using machine learning

# # pd.get_dummies(moviesfrombooks_data, columns=['movie_genre']) DOES NOT WORK

# # adpated from stack overflow
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer(sparse_output=True)

# moviesfrombooks_data = moviesfrombooks_data.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 mlb.fit_transform(moviesfrombooks_data['genre']), #.pop if you want to remove original column of list
#                 index=moviesfrombooks_data.index,
#                 columns=mlb.classes_))

In [23]:
# moviesfrombooks_data.tail()

,title,year,length,genre,stars,gross,rating_N/A,rating_Not Rated,rating_PG,rating_PG-13,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
95,Like Dandelion Dust,2009,100,[Drama],7,"352,810",0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
96,Public Enemies,2009,140,"[Action, Biography, Crime]",7,"97,104,620",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Confessions of a Shopaholic,2009,104,"[Comedy, Romance]",5.8,"44,277,350",0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
98,Inkheart,2008,106,"[Adventure, Family, Fantasy]",6.1,"17,303,424",0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99,Revolutionary Road,2008,119,"[Drama, Romance]",7.3,"22,911,480",0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
# Save/export dataframe, moviesfrombooks_data, to csv file.

moviesfrombooks_data.to_csv('Resources/CLEANEDimdb.csv', index=True)

In [24]:
# Block no longer needed to avoid over-writing original data

# Save/export dataframe, moviesfrombooks_data, to csv file.

# moviesfrombooks_data.to_csv('Resources/CLEANEDimdb_movies_from_books.csv', index=True)